In [1]:
#-*- coding:utf-8 -*-
import numpy as np
import numpy.random as rd
from scipy import stats as st

import matplotlib.pyplot as plt

In [2]:
seed = 0
n = [200, 150, 150] #各データ数
K = 3 #潜在変数の数
D = 2 #次元

#mu:D次元
mu_true = np.array(
    [[0.2, 0.5],
     [1.2, 0.5],
     [2.0, 0.5]])

#sigma: D×D次元
sigma_true = np.array(
    [[[0.1,  0.085], [0.085, 0.1]],
     [[0.1, -0.085], [-0.085, 0.1]],
     [[0.1,  0.085], [0.085, 0.1]]
    ])

rd.seed(seed)
org_data = None
for i in range(K):
    #k_0 に属するデータを生成
    if org_data is None:
        org_data = np.c_[st.multivariate_normal.rvs(mean=mu_true[i], cov=sigma_true[i], size=n[i]), np.ones(n[i])*i]
        
    #k_1, k_2に属するデータを生成し、結合する
    else:
        tmp_data = np.c_[st.multivariate_normal.rvs(mean=mu_true[i], cov=sigma_true[i], size=n[i]), np.ones(n[i])*i]
        org_data = np.r_[org_data, tmp_data]

#print(org_data)

In [3]:
data = org_data[:,:2].copy()
print(data)

[[ -3.71170206e-01  -1.86094478e-03]
 [ -2.91738581e-01   3.96395507e-01]
 [ -2.83360841e-01  -1.52630335e-01]
 [ -7.58501926e-02   1.97933970e-01]
 [  1.95833915e-01   5.66951662e-01]
 [  3.02470780e-02   5.82134638e-01]
 [ -4.19979535e-02   2.79076778e-01]
 [  3.61072236e-02   3.93901312e-01]
 [ -2.36639183e-01   2.78263630e-02]
 [  1.78751036e-01   3.30817315e-01]
 [  9.19856509e-01   1.33306657e+00]
 [  1.36537013e-03   1.72818618e-01]
 [ -3.64367157e-01  -3.16270681e-01]
 [  2.02293687e-01   4.69872493e-01]
 [ -3.93426801e-01   1.61073604e-01]
 [  1.20124745e-01   4.85624415e-01]
 [  6.41551500e-01   5.98467488e-01]
 [  2.92273131e-01   6.19353567e-01]
 [ -2.78307452e-01   2.29950847e-01]
 [  3.43981039e-01   5.91620666e-01]
 [  6.41882095e-01   6.95927774e-01]
 [  5.49999714e-01   1.18788392e+00]
 [  3.92943007e-01   6.17066312e-01]
 [  5.13690194e-01   9.48355474e-01]
 [  7.09271716e-01   9.72424019e-01]
 [  4.38838783e-01   8.05852261e-01]
 [  4.57601064e-01   5.53109571e-01]
 

In [4]:
#Given
T = len(data)
alpha = 1.0
r = 1 / T
k = K #潜在変数の数
D = 2 #次元

In [5]:
print(seed)

0


In [6]:
#初期化
rd.seed(seed)

#空のパラメータを用意
pi = np.zeros((T+1, k))
mu = np.zeros((T+1, k, D))
mu_ = mu.copy()
sigma = np.zeros((T+1, k, D, D))
sigma_ = sigma.copy()

#各パラメータの初期化
for i in range(k):
    pi[0, i] = 1 / k #piの初期化
    mu[0, i] = rd.uniform(low=0, high=1, size=D) #muの初期化(一様分布)
    mu_[0, i] = mu[0, i] * pi[0, i, np.newaxis] #mu_の初期化(mu*piで計算)
    sigma[0, i] = np.eye(D) #sigmaの初期化(単位行列)
    sigma_[0, i] = (sigma[0, i] + np.dot(mu[0, i][:,np.newaxis], mu[0, i][:,np.newaxis].T)) * pi[0, i] #sigma_初期化

print(pi[0])
print(mu[0])
print(mu_[0])
print(sigma[0])
print(sigma_[0])

t = 1

[ 0.33333333  0.33333333  0.33333333]
[[ 0.5488135   0.71518937]
 [ 0.60276338  0.54488318]
 [ 0.4236548   0.64589411]]
[[ 0.18293783  0.23839646]
 [ 0.20092113  0.18162773]
 [ 0.14121827  0.21529804]]
[[[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]]
[[[ 0.43373209  0.13083519]
  [ 0.13083519  0.50383194]]

 [[ 0.45444123  0.10947854]
  [ 0.10947854  0.43229923]]

 [[ 0.39316113  0.09121205]
  [ 0.09121205  0.47239307]]]


In [7]:
#E-Step
def E_step(d, t):
    '''Eステップ(負担率gammaから各パラメータ_とpiを求める)'''
    
    #pi*p(y|mu,sigma)を計算する
    pi_prob = np.array([pi[t-1, i]*st.multivariate_normal.pdf(d, mu[t-1,i], sigma[t-1, i]) for i in range(k)])
    #print(pi_prob)
    
    gamma = (1-alpha*r) * pi_prob / np.sum(pi_prob) + (alpha*r / k)
    #print(gamma)
    
    #piを計算する
    pi[t] = (1-r)*pi[t-1] + r*gamma
    #print(pi[t]) #ok
    
    #mu_を計算する
    mu_[t] = (1-r)*mu_[t-1] + r*gamma[:,np.newaxis]*d
    #print(mu_[t]) #ok
    
    #sigma_を計算する
    for i in range(k):
        sigma_[t,i] = (1-r)*sigma_[t-1,i] + r*gamma[i]*np.dot(d[:,np.newaxis], d[:,np.newaxis].T)         
    #print(sigma_[t])
    

In [8]:
E_step(data[0], 1)

In [9]:
#M-Step
def M_step(d, t):
    '''Mステップ(gammaを使って、各パラメータを更新する)'''
    
    #muを計算する
    mu[t] = mu_[t] / pi[t][:,np.newaxis]
    #print(mu[t]) #ok
    
    #sigmaを計算する
    for i in range(k):
        sigma[t,i] = sigma_[t,i]/pi[t,i] - np.dot(mu[t,i][:,np.newaxis],mu[t,i][:,np.newaxis].T)
    
    #print(sigma[t]) #ok

In [10]:
M_step(data[0], 1)

In [11]:
#パラメータ更新:
while t <= T:
    #print(t)
    y_t = data[t-1]
    #print(y_t)
    
    #E-step
    E_step(y_t, t)
        
    #M-Step
    M_step(y_t, t)
    
    if t % 100 == 0:
        print('t:', t)
        print('mu:', mu[t])
        print('sigma:', sigma[t])
    t += 1

t: 100
mu: [[ 0.48627102  0.68109863]
 [ 0.52906194  0.53894788]
 [ 0.37822213  0.61989204]]
sigma: [[[ 0.85733517  0.02538845]
  [ 0.02538845  0.84518389]]

 [[ 0.8648922   0.01771268]
  [ 0.01771268  0.84042064]]

 [[ 0.84125832  0.02175922]
  [ 0.02175922  0.83555129]]]
t: 200
mu: [[ 0.44608562  0.65636046]
 [ 0.47985939  0.53755699]
 [ 0.35358819  0.6025808 ]]
sigma: [[[ 0.72699698  0.03969627]
  [ 0.03969627  0.71386687]]

 [[ 0.73749984  0.02906699]
  [ 0.02906699  0.70761814]]

 [[ 0.70346879  0.03438729]
  [ 0.03438729  0.6988974 ]]]
t: 300
mu: [[ 0.58677341  0.6211155 ]
 [ 0.62739352  0.51805241]
 [ 0.49920471  0.57973828]]
sigma: [[[ 0.70281132 -0.00648168]
  [-0.00648168  0.61023263]]

 [[ 0.7060085  -0.00636153]
  [-0.00636153  0.59418574]]

 [[ 0.70530992 -0.0031028 ]
  [-0.0031028   0.60187095]]]
t: 400
mu: [[ 0.78203281  0.5941066 ]
 [ 0.83208943  0.50683729]
 [ 0.68693694  0.56107541]]
sigma: [[[ 0.78943263 -0.02467122]
  [-0.02467122  0.51892501]]

 [[ 0.78402657 -0.00

In [12]:
# plot generated data        
plt.figure(figsize=(12, 5))
for i in range(3):
    plt.scatter(org_data[org_data[:,2]==i][:,0], org_data[org_data[:,2]==i][:,1], s=30, alpha=0.1)

c_ = ["blue", "red", "green"]
for i in range(3):
    plt.scatter(mu[0, i, 0], mu[0, i, 1], c=c_[i], s=50)

In [13]:
# plot generated data        
plt.figure(figsize=(12, 5))
for i in range(3):
    plt.scatter(org_data[org_data[:,2]==i][:,0], org_data[org_data[:,2]==i][:,1], s=30, alpha=0.1)

c_ = ["blue", "red", "green"]
for i in range(3):
    plt.scatter(mu[-1, i, 0], mu[-1, i, 1], c=c_[i], s=50)